In [1]:
import astropy
from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib.colors import LogNorm
from astropy import units as u
import galpy
from galpy.orbit import Orbit
from astropy.coordinates import SkyCoord
from galpy.potential import MWPotential2014
from astropy.coordinates import SkyCoord, Galactocentric, CartesianDifferential
from matplotlib.colors import Normalize
from matplotlib.patches import Ellipse, Circle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import pygmmis
from sklearn.cluster import KMeans
import json
import pickle
from galpy.actionAngle import actionAngleStaeckel

  Referenced from: <24C0D3FB-0588-390F-A546-ED36306868E4> /Users/raunaqrai/miniforge3/lib/python3.12/site-packages/galpy/.dylibs/libgfortran.5.dylib
  Reason: tried: '/Users/raunaqrai/miniforge3/lib/python3.12/site-packages/galpy/.dylibs/libquadmath.0.dylib' (duplicate LC_RPATH '@loader_path'), '/usr/local/lib/libquadmath.0.dylib' (no such file), '/usr/lib/libquadmath.0.dylib' (no such file, not in dyld cache)'


In [14]:
vphi_data = '../data/df_v_final.fits'

with fits.open(vphi_data) as hdul:
    data = hdul[1].data  
    
    df_v = pd.DataFrame({col.name: data[col.name].byteswap().newbyteorder() if data[col.name].dtype.byteorder == '>' else data[col.name]
                       for col in hdul[1].columns})
    
pd.set_option('display.max_columns', None)

df_v = df_v[np.abs(df_v['Z']) < 2.5]
df_v = df_v.dropna(subset=["v_R", "v_phi", "v_R_uncertainty", "v_phi_uncertainty", "v_Z", "v_Z_uncertainty"])

df = df_v.copy()

In [15]:
df.head()

,source_id,l,b,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ruwe,radial_velocity,radial_velocity_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,catwise_w1,catwise_w2,mh_xgboost,teff_xgboost,logg_xgboost,in_training_sample,col1,col2,Source,RA_ICRS,DE_ICRS,rgeo,b_rgeo_x,B_rgeo_xa,rpgeo,b_rpgeo_x,B_rpgeo_xa,Flag,angDist,fpu,E(B-V),distance_kpc,R,Z,R_gal,phi_gal,Z_gal,v_phi,v_R,v_Z,rpgeo_error,v_R_uncertainty,v_phi_uncertainty,v_Z_uncertainty
0,15741055975040,176.739184,-48.572035,45.136038,0.335043,1.439792,0.018947,-0.711280,0.017718,-1.412098,0.016528,1.036041,-0.738894,0.316921,10.254021,10.750235,9.595748,8.152,8.198,-0.144,5065.8,2.993,True,45.136038,0.335043,15741055975040,45.136038,0.335043,695.683899,683.627625,707.396423,696.278320,688.270874,707.143982,10033,0.0,0.013160,0.104689,696.278320,8.582006,-0.522061,470.485258,0.055731,-520.632142,243.095762,-14.428048,4.529900,9.436554,0.205318,0.064645,0.226755
1,66627828480768,176.483565,-48.171322,45.305053,0.736093,0.534038,0.020692,3.309832,0.022959,1.594356,0.022822,1.349013,41.607450,1.323430,10.508830,11.150953,9.741709,7.891,7.964,-0.401,4499.0,1.916,False,45.305053,0.736093,66627828480768,45.305053,0.736093,1884.275020,1821.247560,1947.302730,1883.143550,1807.318600,1944.665770,10033,0.0,0.038747,0.093026,1883.143550,9.375831,-1.403210,1268.325930,0.060773,-1399.358389,237.364263,42.346437,-2.860027,68.673585,1.199227,0.372403,1.029146
2,82467667849472,176.209301,-48.607026,44.866246,0.561503,2.209985,0.016049,-4.869755,0.015797,-12.678339,0.013950,1.018742,-32.461674,0.205614,10.651456,11.169626,9.980610,8.496,8.558,0.114,4938.3,3.203,True,44.866246,0.561503,82467667849472,44.866246,0.561503,452.636078,448.701294,456.615540,452.602692,449.257355,455.432892,10033,0.0,0.007262,0.086075,452.602692,8.420638,-0.339539,308.483632,0.064184,-338.602406,230.329127,-51.045366,13.944756,3.087768,0.194034,0.100426,0.199619
3,101193725229056,175.755174,-48.727781,44.569524,0.689953,0.528788,0.024241,3.539184,0.027864,-1.599436,0.021891,1.176748,91.295900,4.830661,14.332739,14.816101,13.685076,12.220,12.275,-0.478,4980.2,3.319,False,44.569524,0.689953,101193725229056,44.569524,0.689953,1908.478270,1815.024170,2014.483890,1868.603880,1800.940800,1949.831300,10033,0.0,0.045842,0.078062,1868.603880,9.351665,-1.404413,1245.041147,0.073347,-1400.635394,217.472712,62.829005,-53.068842,74.445250,3.110205,1.267245,3.430299
4,130399502833792,175.789759,-48.328584,44.868872,0.955080,0.847929,0.017479,3.382907,0.017619,-0.291235,0.015982,1.087555,1.959265,0.328298,11.215295,11.810178,10.481738,8.728,8.815,-0.159,4643.6,2.398,True,44.868872,0.955080,130399502833792,44.868872,0.955080,1178.407470,1155.319820,1203.711180,1177.687260,1154.219600,1201.322750,10033,0.0,0.020614,0.077163,1177.687260,8.903067,-0.879697,793.842786,0.072479,-877.288236,231.337541,1.866465,13.969564,23.551575,0.309798,0.298155,0.315210


In [22]:
import numpy as np
import pandas as pd
from galpy.potential import MWPotential2014, evaluatePotentials
from astropy import units as u

# Load data
R = df['R'].values * u.kpc
Z = df['Z'].values * u.kpc
phi = np.deg2rad(df['phi_gal'].values)  # convert to radians if not already

v_R = df['v_R'].values * u.km / u.s
v_phi = df['v_phi'].values * u.km / u.s
v_Z = df['v_Z'].values * u.km / u.s

# Convert to Cartesian coordinates
X = R * np.cos(phi)
Y = R * np.sin(phi)

v_X = v_R * np.cos(phi) - v_phi * np.sin(phi)
v_Y = v_R * np.sin(phi) + v_phi * np.cos(phi)

# Compute specific angular momentum components (in kpc km/s)
L_x = (Y * v_Z - Z * v_Y).to(u.kpc * u.km / u.s)
L_y = (Z * v_X - X * v_Z).to(u.kpc * u.km / u.s)
L_z = (X * v_Y - Y * v_X).to(u.kpc * u.km / u.s)
L_perp = np.sqrt(L_x**2 + L_y**2)
L_tot = np.sqrt(L_x**2 + L_y**2 + L_z**2)

# Compute kinetic energy per unit mass (in km²/s²)
E_kin = 0.5 * (v_R**2 + v_phi**2 + v_Z**2)

# Compute potential energy at each (R, Z)
Phi = evaluatePotentials(MWPotential2014, R, Z)

from galpy.util import conversion

# Conversion factor: potential (galpy units) → (km/s)^2
pot_to_kms2 = conversion.get_potential_in_velocity2(MWPotential2014)

# Convert Phi to km²/s²
Phi_kms2 = Phi * pot_to_kms2 * (u.km/u.s)**2

# Total specific energy
E_tot = E_kin + Phi_kms2



# Add to DataFrame
df['L_x'] = L_x.value
df['L_y'] = L_y.value
df['L_z'] = L_z.value
df['L_perp'] = L_perp.value
df['L_tot'] = L_tot.value
df['E_kin'] = E_kin.value
df['Phi'] = Phi.value
df['E_tot'] = E_tot.value

print("✅ Computed IoM: L_x, L_y, L_z, L_tot, E_kin, Phi, E_tot")


AttributeError: module 'galpy.util.conversion' has no attribute 'get_potential_in_velocity2'